In [1]:
import os
import pandas as pd

In [2]:
RESULTS_DIR = "/data/pinello/PROJECTS/2023_08_ZL/kaggle_scp/results/deep_tensor_factorization"
DE_TRAIN = "/data/pinello/PROJECTS/2023_08_ZL/kaggle_scp/data/de_train.parquet"
ID_MAP_CSV = "/data/pinello/PROJECTS/2023_08_ZL/kaggle_scp/data/id_map.csv"

In [3]:
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

In [4]:
df = pd.read_parquet(DE_TRAIN)

In [5]:
df.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [6]:
df['cell_type'].unique().tolist()

['NK cells',
 'T cells CD4+',
 'T cells CD8+',
 'T regulatory cells',
 'B cells',
 'Myeloid cells']

In [7]:
# Create training and validate datasets 
# Here, for each cell types in NK cells, T CD4+, T reg, and T CD8+, we
# use the test compounds as valication dataset
df_id_map = pd.read_csv(ID_MAP_CSV)
test_compunds = df_id_map['sm_name'].unique()

#df_id_map.head()

In [8]:
def convert_to_long_df(df):
    df = df.drop(['sm_lincs_id', 'SMILES', 'control'], axis=1)
    df = pd.melt(df, id_vars=['cell_type', 'sm_name'], var_name='gene', value_name='target')
    
    return df

In [9]:
cell_type_names = {'NK cells': 'nk',
                   'T cells CD4+': 't_cd4',
                   'T cells CD8+': 't_cd8',
                   'T regulatory cells': 't_reg'}

In [10]:
for key, value in cell_type_names.items():
    print(key)
    df_train = df[(df['cell_type'] != key) | ~df['sm_name'].isin(test_compunds)]
    df_valid = df[(df['cell_type'] == key) & df['sm_name'].isin(test_compunds)]
    
    df_train = df_train.sort_values(['cell_type', 'sm_name'])
    df_valid = df_valid.sort_values('sm_name')
    
    df_train = convert_to_long_df(df_train)
    df_valid = convert_to_long_df(df_valid)
    
    df_train.to_csv(f'{RESULTS_DIR}/df_train_{value}.csv')
    df_valid.to_csv(f'{RESULTS_DIR}/df_valid_{value}.csv')

NK cells
T cells CD4+
T cells CD8+
T regulatory cells
